In [ ]:
!pip install transformers datasets torch accelerate gradio -q

In [1]:
import os
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "0"   # allow model download, but no login
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["WANDB_DISABLED"] = "true"      # disables wandb tracking
os.environ["HF_HOME"] = "/tmp"             # use temp storage
os.environ["HF_DATASETS_OFFLINE"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
#  Imports
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, pipeline
from datasets import Dataset
import gradio as gr

In [3]:
# Create a small instruction dataset
data = [
    {"instruction": "Summarize the given text.",
     "input": "Machine learning is a field of artificial intelligence that enables systems to learn from data and improve automatically.",
     "output": "Machine learning enables systems to learn and improve from data."},
    {"instruction": "Explain what tokenization means in NLP.",
     "input": "",
     "output": "Tokenization is the process of splitting text into smaller units like words or subwords."},
    {"instruction": "Define embedding in simple terms.",
     "input": "",
     "output": "An embedding is a way to represent words as numerical vectors capturing their meaning."},
    {"instruction": "Summarize the text.",
     "input": "Artificial Intelligence enables machines to perform tasks that normally require human intelligence.",
     "output": "AI enables machines to mimic human intelligence."},
]
dataset = Dataset.from_list(data)

In [4]:
# Tokenization / preprocessing
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name, use_auth_token=False)

def preprocess(example):
    inp = f"Instruction: {example['instruction']} Input: {example['input']}"
    model_inputs = tokenizer(inp, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset.map(preprocess, batched=False)


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1908: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the 

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [5]:
#  Model & training (fine-tuning on small data)
model = T5ForConditionalGeneration.from_pretrained(model_name, use_auth_token=False)

training_args = TrainingArguments(
    output_dir="./t5_instruction_finetune",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no",
    push_to_hub=False,        # <— prevents token prompt
    report_to=[]              # <— disables any reporting
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_data)
trainer.train()
trainer.save_model("./t5_instruction_model")

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4841: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Step,Training Loss


In [6]:
#  Inference pipeline
pipe = pipeline("text2text-generation", model="./t5_instruction_model", tokenizer=model_name)

Device set to use cuda:0


In [7]:

#  Gradio Interface
def generate_text(instruction, user_input):
    prompt = f"Instruction: {instruction} Input: {user_input}"
    result = pipe(prompt, max_length=80)[0]['generated_text']
    return result

In [8]:
demo = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Instruction (e.g. Summarize the text)"),
            gr.Textbox(label="Input Text")],
    outputs=gr.Textbox(label="Generated Output"),
    title=" Mini GenAI Instruction Model (Token-Free)",
    description="Fine-tuned T5-small model trained "
)

In [9]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0014b938b07176b97c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
